In [ ]:
import requests
import os
import pandas as pd

#### Purpose:
Make one master conversion county - zip file

In [ ]:
hud_url = 'https://www.huduser.gov/portal/datasets/usps/COUNTY_ZIP_'
folder = 'geo_transform_data'
os.mkdir(folder)

In [ ]:
r = requests.get(hud_url + '032020' + '.xlsx')

In [ ]:
r.status_code == 200

In [ ]:
def file_download(month_year,data_add = pd.DataFrame()):
    '''
    For a given month during a given year (supplied as one string: 'MMYYYY')
    download the corresponding county to zip data, save it locally to the folder specified above.
    Also adds the data to a given dataframe (data_add), which can be saved as one file later on. 
    '''
    file = folder + '/' + month_year + '.xlsx'
    r = requests.get(hud_url + month_year + '.xlsx')
    if r.status_code == 200:
        with open(file, 'wb') as f:
            for chunk in r.iter_content(chunk_size=1024): 
                if chunk: # filter out keep-alive new chunks
                    f.write(chunk)
        df = pd.read_excel(file)
        df['date'] = month_year
        #upper case all columns
        df = df.rename({x:x.upper() for x in df.columns}, axis = 1)
        if not data_add.empty:
            data = pd.concat([df,data_add],axis = 0)
        else:
            data = df
        return(data)
    return(data_add)

In [ ]:
months = ['03','06','09','12']
years = ['2010','2011','2012','2013','2014','2015','2016','2017','2018','2019','2020']*4
years.sort()
dates = [a+b for (a,b) in zip(months*11,years)]

In [ ]:
data = file_download(dates[0])

In [ ]:
for date in dates[1:]:
    data = file_download(date,data_add = data)

In [ ]:
data = data.drop(' ',axis = 1)

In [ ]:
data.to_csv(folder + '/county_zip_all.csv')